In [20]:
#convert YOLO predictions to FiftyOne dataset

import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F
from ultralytics import YOLO
import numpy as np
import os
from tqdm import tqdm

# name = 'dataset_demo2'
# dataset_dir = '/Volumes/Cara_camera_traps/CV4E/data_cleaned/demo2'

In [8]:
#practice. I have run inference and have a folder of predictions in YOLO .txt format
#code from here: https://docs.voxel51.com/tutorials/yolov8.html#Fine-tune-YOLOv8-models-for-custom-use-cases-with-the-help-of-FiftyOne

#load a single file
label_file = '/Volumes/Cara_cam_traps 1/CV4E/data_cleaned/demo2/predict/labels/20266-1__20266-1-G__2021-03-31__21-26-08(1)_gw.txt'
with open(label_file) as f:
    print(f.read())    

15 0.848947 0.391104 0.302106 0.772856 0.442625



In [21]:
#make predictions using yolo model
detection_model = YOLO('yolov8n.pt')
dataset = foz.load_zoo_dataset(
    'coco-2017',
    split='validation',
)
coco_classes = [c for c in dataset.default_classes if not c.isnumeric()]

#load sample fiftyone dataset
def export_yolo_data(
    samples,
    export_dir,
    classes,
    label_field = "ground_truth",
    split = None
    ):

    if type(split) == list:
        splits = split
        for split in splits:
            export_yolo_data(
                samples,
                export_dir,
                classes,
                label_field,
                split
            )
    else:
        if split is None:
            split_view = samples
            split = "val"
        else:
            split_view = samples.match_tags(split)

        split_view.export(
            export_dir=export_dir,
            dataset_type=fo.types.YOLOv5Dataset,
            label_field=label_field,
            classes=classes,
            split=split
        )

100%|██████████| 6.23M/6.23M [00:00<00:00, 29.8MB/s]


 100% |██████|    1.9Gb/1.9Gb [10.0s elapsed, 0s remaining, 203.8Mb/s]      
Extracting annotations to '/Users/caraappel/fiftyone/coco-2017/raw/instances_val2017.json'
 100% |██████|    6.1Gb/6.1Gb [30.8s elapsed, 0s remaining, 240.1Mb/s]      
Extracting images to '/Users/caraappel/fiftyone/coco-2017/validation/data'
Writing annotations to '/Users/caraappel/fiftyone/coco-2017/validation/labels.json'
Dataset info written to '/Users/caraappel/fiftyone/coco-2017/info.json'
Loading 'coco-2017' split 'validation'
 100% |███████████████| 5000/5000 [17.0s elapsed, 0s remaining, 292.9 samples/s]      
Dataset 'coco-2017-validation' created


In [22]:
coco_val_dir = "coco_val"
export_yolo_data(dataset, coco_val_dir, coco_classes)
#Then run inference on these images:

!yolo task=detect mode=predict model=yolov8n.pt source=coco_val/images/val save_txt=True save_conf=True

 100% |███████████████| 5000/5000 [11.6s elapsed, 0s remaining, 480.6 samples/s]      
Ultralytics YOLOv8.0.199 🚀 Python-3.8.17 torch-2.1.0 CPU (Apple M2 Pro)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

image 1/5000 /Users/caraappel/Documents/CV4E/oregon_critters/scripts/4_post/coco_val/images/val/000000000139.jpg: 448x640 1 person, 5 chairs, 1 potted plant, 2 dining tables, 1 tv, 1 refrigerator, 1 clock, 1 vase, 65.9ms
image 2/5000 /Users/caraappel/Documents/CV4E/oregon_critters/scripts/4_post/coco_val/images/val/000000000285.jpg: 640x608 1 bear, 75.1ms
image 3/5000 /Users/caraappel/Documents/CV4E/oregon_critters/scripts/4_post/coco_val/images/val/000000000632.jpg: 512x640 1 bottle, 1 chair, 2 potted plants, 1 bed, 2 books, 64.6ms
image 4/5000 /Users/caraappel/Documents/CV4E/oregon_critters/scripts/4_post/coco_val/images/val/000000000724.jpg: 640x480 1 truck, 2 stop signs, 56.0ms
image 5/5000 /Users/caraappel/Documents/CV4E/oregon_critters/scripts

In [17]:


#read all predictions in this file as numpy array
def read_yolo_detections_file(filepath):
    detections = []
    if not os.path.exists(filepath):
        return np.array([])

    with open(filepath) as f:
        lines = [line.rstrip('\n').split(' ') for line in f]

    for line in lines:
        detection = [float(l) for l in line]
        detections.append(detection)
    return np.array(detections)

##TEST IT
yolo_file = read_yolo_detections_file(label_file)
print(yolo_file)

#convert bbox coordinates from YOLO format (center x/y) to FiftyOne format (top left x/y)
def _uncenter_boxes(boxes):
    '''convert from center coords to corner coords'''
    boxes[:, 0] -= boxes[:, 2]/2.
    boxes[:, 1] -= boxes[:, 3]/2.

#convert list of class prediction indices to list of class labels
def _get_class_labels(predicted_classes, class_list):
    labels = (predicted_classes).astype(int)
    labels = [class_list[l] for l in labels]
    return labels

##TEST IT
class_list = 

[[         15     0.84895      0.3911     0.30211     0.77286     0.44262]]


In [11]:
#use all this to convert YOLO detections to FiftyOne format
def convert_yolo_detections_to_fiftyone(
    yolo_detections,
    class_list
    ):

    detections = []
    if yolo_detections.size == 0:
        return fo.Detections(detections=detections)

    boxes = yolo_detections[:, 1:-1]
    _uncenter_boxes(boxes)

    confs = yolo_detections[:, -1]
    labels = _get_class_labels(yolo_detections[:, 0], class_list)

    for label, conf, box in zip(labels, confs, boxes):
        detections.append(
            fo.Detection(
                label=label,
                bounding_box=box.tolist(),
                confidence=conf
            )
        )

    return fo.Detections(detections=detections)

In [12]:
#finally, take in an image path and return file path of corresponding YOLOv8 detection prediction file
def get_prediction_filepath(filepath, run_number = 1):
    run_num_string = ""
    if run_number != 1:
        run_num_string = str(run_number)
    filename = filepath.split("/")[-1].split(".")[0]
    return f"runs/detect/predict{run_num_string}/labels/{filename}.txt"

In [ ]:
#put all together to do in batch
def add_yolo_detections(
    samples,
    prediction_field,
    prediction_filepath,
    class_list
    ):

    prediction_filepaths = samples.values(prediction_filepath)
    yolo_detections = [read_yolo_detections_file(pf) for pf in prediction_filepaths]
    detections =  [convert_yolo_detections_to_fiftyone(yd, class_list) for yd in yolo_detections]
    samples.set_values(prediction_field, detections)

In [ ]:
#now get detections!

filepaths = dataset.values("filepath")

prediction_filepaths = [get_prediction_filepath(fp) for fp in filepaths]
dataset.set_values(
    "yolov8n_det_filepath",
    prediction_filepaths
)

add_yolo_detections(
    dataset,
    "yolov8n",
    "yolov8n_det_filepath",
    coco_classes
)

In [ ]:
#old way...?

#splits to load
splits = ['train','val']

#load dataset
dataset = fo.Dataset(name)
for split in splits:
    dataset.add_dir(
        dataset_dir = dataset_dir,
        dataset_type = fo.types.YOLOv5Dataset,
        split = split,
        tags = split,
    )